In [1]:
import gspread
from google.oauth2.service_account import Credentials
import re
import configparser
import pandas as pd
import uuid
import sqlite3

In [2]:
# return list of source sheets
def source_sheets(contact_config):
    sheet_info = []
    for key in contact_config.keys():
        if 'source_sheet' in key:
            sheet, column, row = contact_config.get(key).split(',')
            sheet_info.append( (sheet, int(column), int(row)) )
    return tuple(sheet_info)

# split emails if they put two in a cell
def split_emails(raw_list):
    split_result = []
    for entry in raw_list:
        # Split on anything that isn't a valid email character
        parts = re.split(r'[^a-zA-Z0-9._%+\-@]+', entry)
        split_result.extend([p.strip() for p in parts if p.strip()])
    return split_result

# Basic email pattern: username@domain
EMAIL_REGEX = re.compile(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$")
def is_valid_email(email):
    return EMAIL_REGEX.match(email) is not None

# builds a vcard
def generate_vcard(first, last, fn, mobile, email, notes):
    return f"""BEGIN:VCARD
VERSION:3.0
N:{last};{first};;;
FN:{fn}
TEL;TYPE=CELL:{mobile}
EMAIL:{email}
NOTE:{notes}
END:VCARD
"""

In [3]:
# get config
cp = configparser.ConfigParser() 
cp.read('config.ini')
config = dict(cp['contacts'])

# Setup auth and client
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file(config.get('credentials'), scopes=SCOPES)
client = gspread.authorize(creds)

In [4]:
# Open your spreadsheet by name or URL
spreadsheet = client.open(config.get('workbook'))
records = {}

In [5]:
data = spreadsheet.worksheet("AGIMBA Contacts").get_values("A1:E")
# Convert to list of dicts
keys = data[0]
rows = data[1:]

for contact in [dict(zip(keys, row)) for row in rows]:
    email = contact.get('email').lower()
    if not is_valid_email(email):
        continue
    records[email] = {'email': email,
                                'first_name': contact.get('first_name'),
                                'last_name': contact.get('last_name'),
                                'formatted_name': f"{contact.get('first_name')} {contact.get('last_name')}",
                                'phone': contact.get('phone'),
                                'notes': '',
                               }

In [6]:
data = spreadsheet.worksheet("AGMB Roster").get_values("A1:F")
# Convert to list of dicts
keys = ['student_first_name', 'student_last_name', 'grade', 'section', 'parent_name', 'email']
rows = data[1:]

for contact in [dict(zip(keys, row)) for row in rows]:
    raw_email = contact.get('email').lower()
    for email in split_emails([raw_email,]):
        if not is_valid_email(email):
            continue
        if not records.get(email):
            records[email] = {'email': email,
                                        'first_name': contact.get('parent_name'),
                                        'last_name': '',
                                        'formatted_name': contact.get('parent_name'),
                                        'phone': '',
                                        'notes': '',
                                       }    
        records[email]['notes'] = '\n'.join([ f"{records.get(email).get('notes')}",
             f"Student: {contact.get('student_first_name').strip()} {contact.get('student_last_name').strip()} -> {contact.get('section').strip()}"] )

In [7]:
data = spreadsheet.worksheet("AGIMBA Org").get_values("A1:B")
# Convert to list of dicts
keys = ['role', 'email']
rows = data[1:]

for contact in [dict(zip(keys, row)) for row in rows]:
    for email in split_emails([contact.get('email').lower(),]):
        if not is_valid_email(email):
            continue
        if not records.get(email):
            records[email] = {'email': email,
                                        'first_name': contact.get('role'),
                                        'last_name': '',
                                        'formatted_name': contact.get('role'),
                                        'phone': '',
                                        'notes': '',
                                       }

In [8]:
data = spreadsheet.worksheet("AG Staff").get_values("A1:C")
# Convert to list of dicts
keys = ['first_name', 'last_name', 'email']
rows = data[1:]

for contact in [dict(zip(keys, row)) for row in rows]:
    for email in split_emails([contact.get('email').lower(),]):
        if not is_valid_email(email):
            continue
        if not records.get(email):
            records[email] = {'email': email,
                                        'first_name': contact.get('first_name'),
                                        'last_name': contact.get('last_name'),
                                        'formatted_name': f"{contact.get('first_name')} {contact.get('last_name')}",
                                        'phone': '',
                                        'notes': '',
                                       }

In [9]:
with open('contacts.vcf', 'w') as vcf_file:
    for key in sorted(records):
        vcard = generate_vcard(
            records.get(key).get('first_name'),
            records.get(key).get('last_name'),
            records.get(key).get('formatted_name'),
            records.get(key).get('phone'),
            records.get(key).get('email'),
            records.get(key).get('notes')
        )
        vcf_file.write(vcard + '\n')
        #print(vcard)

In [10]:
# Build list of lists as need for sheet update
update_list = [ [email,] for email in sorted(records.keys()) ]

# Clear old data
sheet_email_lists = spreadsheet.worksheet(config.get('email_sheet'))
sheet_email_lists.batch_clear([config.get('email_range')])

# Prepare for writing
sheet_email_lists.update(range_name=f'{config.get('email_range')}{len(update_list)+1}', values=update_list )

print(f"Wrote {len(update_list)} unique emails to '{sheet_email_lists.title}' sheet.")

Wrote 110 unique emails to 'EmailLists' sheet.


In [11]:
wanted_keys = ["first_name", "last_name", "email", "phone"]

result = [
    {k: v for k, v in inner.items() if k in wanted_keys}
    for inner in records.values()
]

pd.DataFrame(result).to_csv("agimba_volunteers.csv", index=False)

In [12]:
contact_list = [records.get(contact) for contact in records]

In [13]:
data = spreadsheet.worksheet("AGMB Roster").get_values("A1:F")
# Convert to list of dicts
keys = ['student_first_name', 'student_last_name', 'grade', 'section', 'parent_name', 'email']
rows = data[1:]
roster = [dict(zip(keys, row)) for row in rows]

In [14]:
df = pd.DataFrame(roster)
df['grade'] = pd.to_numeric(df['grade'], errors='coerce')
df['grad_year'] = 2026 + (12 - df['grade'])
df['activity'] = 'Marching Band'
df['first_name'] = df['student_first_name']
df['last_name'] = df['student_last_name']
df['parent_email'] = df['email']
df['activity_year'] = 2025
df['student_id'] = [str(uuid.uuid4()) for _ in range(len(df))]


# Connect to SQLite file
with sqlite3.connect('agimba.db') as conn:
    # student table
    columns_to_save = ['first_name', 'last_name', 'grad_year', 'student_id']
    df[columns_to_save].to_sql('students', conn, if_exists='replace', index=False)

    # Save to parents table
    columns_to_save = ['parent_name', 'parent_email']
    df_parents = df[columns_to_save].drop_duplicates()
    df_parents['parent_id'] = [str(uuid.uuid4()) for _ in range(len(df_parents))]
    df_parents.to_sql('parents', conn, if_exists='replace', index=False)

    # Save to activities table
    columns_to_save = ['student_id', 'activity_year', 'activity', 'section']
    df_activities = df[columns_to_save].drop_duplicates()
    df_activities.to_sql('activities', conn, if_exists='replace', index=False)

    # save to student_partent_mapping
    df_mapping = df.merge(df_parents[['parent_email', 'parent_id']], on="parent_email", how="left")
    columns_to_save = ['student_id', 'parent_id']
    df_mapping = df_mapping[columns_to_save].drop_duplicates()
    df_mapping.to_sql('student_parent_mapping', conn, if_exists='replace', index=False)

In [15]:
df_mapping

,student_id,parent_id
0,4f653ac3-00f2-40e2-9ee2-4e5eb5189456,898525c8-796c-4f25-8b46-e8e3d4fd66ce
1,983c3df9-4464-4dd0-8adf-2ac8c2464909,2a93f78a-5be0-46bb-8722-604de828179c
2,5e999cf5-6d54-48bd-9a7b-d5d446355be5,f87e40fd-2f65-49d2-bd8d-047cca80c695
3,28f0eb69-9129-4b24-a3c1-63e209b8d0f8,f866a6bf-b6bc-4096-81a5-948c20996ba7
4,80e6ff2d-13e2-4c52-ba93-c790f32636ee,de861a2f-4613-4df4-928c-9c3c41251126
...,...,...
71,62d3c299-a74b-436e-addf-8ba1843bd5f9,9df5a879-5b6e-479c-8875-35f20cab6330
72,8a7e0178-b9c8-4d1d-99c2-0019fc9835c9,bb10ca98-788e-4928-96ed-2cb237839d89
73,c01c4f78-c60f-41b8-8592-85b371b35c56,8cbb79d2-d057-483c-8f41-f6360396a1a4
74,806ef08e-4066-42a7-b60f-c627ac107e71,df28ddb0-c070-4554-a98c-64cc328bce4a
